Calibration and configuration
=============================

A standard RGB computer monitor has only three primaries. This means it isn't great for silent substitution, but it is a good example to show how to calibrate and configure a multiprimary system for use with *PySilSub* because the same principles can be applied to a device with any number of primaries. 

To make a calibration file you will need to measure the output of the primaries with a spectrometer. As a minimum, the file should include measurements for each primary at its maximum output, as well as an ambient spectral measurement where all primaries are off. Most standard monitors have 8-bit resolution which means that the input is specified with values between 0 and 255. So the settings to be measured are:

* Red: [255, 0, 0]
* Green: [0, 255, 0]
* Blue: [0, 0, 255] 
* Ambient measurement: [0, 0, 0]

One way to obtain these measurements would be to write a simple program that sets the full screen colour using each of the above settings before sampling with a spectrometer. 

When you have the measurements, put them in a CSV file where the first row describes the wavelength sampling (e.g., 380, 381, ..., 780) and every subsequent row contains a spectral measurement. Also include column headers *Primary* and *Setting* with numerical values to identify the spectral measurements. 


| Primary | Setting | 380 | 381 | ... | 780 |
| ------- |-------- | --- | --- | --- | --- |
| 0       | 0       | ... | ... | ... | ... |
| 0       | 255     | ... | ... | ... | ... |
| 1       | 0       | ... | ... | ... | ... |
| 1       | 255     | ... | ... | ... | ... |
| 2       | 0       | ... | ... | ... | ... |
| 2       | 255     | ... | ... | ... | ... |

With the above calibration file we are ready to go:

```Python
from pysilsub.device import StimulationDevice

device = StimulationDevice(
    resolutions = [255, 255, 255],
    colors = ['blue', 'green', 'red'],
    calibration = './calibration_file.csv',
    wavelengths = [380, 780, 1],
    name = 'Standard Monitor'
    )
```

We can now do useful things like plot the calibration SPDs and their chromaticities, as well as predict the spectral output of the device for any combination of settings. 

This is OK, but most computer monitors--and most light devices for that matter--do not have a linear input-output relationship (sometimes called gamma), so our predictions are unlikely to be accurate. We can improve the accuracy of the device model by obtaining multiple measurements for each primary accross the input range. For example:

| Primary | Setting | 380 | 381 | ... | 780 |
| ------- |-------- | --- | --- | --- | --- |
| 0       | 0       | ... | ... | ... | ... |
| 0       | 7       | ... | ... | ... | ... |
| 0       | 15      | ... | ... | ... | ... |
| 0       | ...     | ... | ... | ... | ... |
| 0       | 247     | ... | ... | ... | ... |
| 0       | 255     | ... | ... | ... | ... |
| 1       | 0       | ... | ... | ... | ... |
| 1       | 7       | ... | ... | ... | ... |
| 1       | 15      | ... | ... | ... | ... |
| 1       | ...     | ... | ... | ... | ... |
| 1       | 247     | ... | ... | ... | ... |
| 1       | 255     | ... | ... | ... | ... |
| 2       | 0       | ... | ... | ... | ... |
| 2       | 7       | ... | ... | ... | ... |
| 2       | 15      | ... | ... | ... | ... |
| 2       | ...     | ... | ... | ... | ... |
| 2       | 247     | ... | ... | ... | ... |
| 2       | 255     | ... | ... | ... | ... |

For convenience, you can save your device settings in a JSON file and 

```Python
import json

# Configure device
RESOLUTIONS = [255, 255, 255]
COLORS = ["blue", "green", "red"]
CALIBRATION_FPATH = (
    "~/long/path/to/calibration_file.csv"
)
CALIBRATION_UNITS = "W/m$^2$/nm"
NAME = "Standard Monitor (8-bit, RGB)"
JSON_NAME = "StandardMonitor"
WAVELENGTHS = [380, 781, 1]

# Optional
NOTES = (
    "A standard computer monitor calibration. Not very useful for silent substitution, but we may be able to do something interesting if we ignore rods and melanopsin."
)


def device_config():

    config = {
        "calibration_fpath": CALIBRATION_FPATH,
        "calibration_units": CALIBRATION_UNITS,
        "name": NAME,
        "json_name": JSON_NAME,
        "wavelengths": WAVELENGTHS,
        "colors": COLORS,
        "resolutions": RESOLUTIONS,
        "notes": NOTES,
    }

    json.dump(config, open(f"../data/{JSON_NAME}.json", "w"), indent=4)


if __name__ == "__main__":
    device_config()
```